In [1]:
import torch
import config
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as TF

from net import Net
from enum import Enum
from time import time
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from landmarksDataset import LandmarksDataset

In [2]:
class Modes(Enum):
    TRAIN = 'train'
    TEST = 'test'

In [3]:
def normalizeDataByMinMax(data, mins=None, maxs=None):
    min_history = []
    max_history = []

    for i in range(data.shape[1]):
        # Compute standard deviation
        if (maxs is None or mins is None):
            min_val = torch.min(data[:, i])
            max_val = torch.max(data[:, i])
        else:
            min_val = mins[i]
            max_val = maxs[i]

        denominator = max_val - min_val
        denominator = 1e-7 if denominator == 0 else denominator

        # Save the min and max history for denormalize later
        min_history.append(min_val)
        max_history.append(max_val)

        # Normalize the data
        data[:, i] = (data[:, i] - min_val) / denominator
        
    history = (min_history, max_history)
    return data, history

In [4]:
def normalizeDataByStdMean(data, stds=None, means=None):
    std_history = []
    mean_history = []

    for i in range(data.shape[1]):
        # Compute standard deviation
        if (stds is None or means is None):
            std = torch.std(data[:, i])
            mean = torch.mean(data[:, i])
        else:
            std = stds[i]
            mean = means[i]

        std = 1e-7 if std == 0 else std

        # Save the std and mean history for denormalize later
        std_history.append(std)
        mean_history.append(mean)

        # Normalize the data
        data[:, i] = (data[:, i] - mean) / std
        
    history = (std_history, mean_history)
    return data, history

In [5]:
def getDataFromCSV(mode, file, start, end):
    return pd.read_csv(f'{config.samples_path}\\{mode}\\{file}',
                               usecols = range(start, end))

In [6]:
def getData(mode, device, x_history=None, y_history=None):
    # Get data from csv files
    x = getDataFromCSV(mode, config.x_data_file, 
                        config.x_cols_start_index, config.x_cols_end_index)
                        
    y = getDataFromCSV(mode, config.y_data_file, 
                        config.y_cols_start_index, config.y_cols_end_index)

    # Transforms the data to tensors
    x_tensor = torch.tensor(x.values, requires_grad=True, device = device).float()
    y_tensor = torch.tensor(y.values, requires_grad=True, device = device).float()

    # Normallize the data
    norm_x_tensor, x_history = normalizeDataByMinMax(x_tensor, x_history[0] if x_history is not None else None, x_history[1] if x_history is not None else None)
    norm_y_tensor, y_history = normalizeDataByMinMax(y_tensor, y_history[0] if y_history is not None else None, y_history[1] if y_history is not None else None)

    return norm_x_tensor, norm_y_tensor, x_history, y_history

In [7]:
def getDataset(mode, device, x_history=None, y_history=None):
    x, y, x_history, y_history = getData(mode, device, x_history, y_history)
    # if (mode == Modes.TRAIN.value):
    #     print(x.min(axis=0), x.max(axis=0), x.mean(axis=0), x.std(axis=0))
    return LandmarksDataset(x, y), x_history, y_history

In [8]:
def train(device, epochs, train_loader, net, optimizer, criterion, log_interval, test_loader):
    for epoch in range(epochs):
        net.train()
        start = time()
        epoch_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            epoch_loss += loss
            loss.backward()
            optimizer.step()
            # if batch_idx % log_interval == 0:
            #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #         epoch, batch_idx * len(data), len(train_loader.dataset),
            #                100. * batch_idx / len(train_loader), epoch_loss.item() / (batch_idx+1)))
        _, test_loss = test(device, test_loader ,net ,criterion)
        print("train all dataset losss: ", epoch_loss.item())
        print("division by number of examples: ", len(train_loader.dataset))
        print(f'Epoch {epoch + 1} summary- train: {(epoch_loss.item() / len(train_loader.dataset)) * 100:.5f} test: {test_loss :.5f} took {time()-start:.2f}secs')
    return net

In [9]:
def test(device, test_loader, net, criterion):
    net.eval()
    with torch.no_grad():
        test_loss = 0
        # correct = 0
        results = torch.tensor([], device = device).float()

        for data, target in test_loader:
            data, target = Variable(data), Variable(target)
            net_out = net(data)
            results = torch.cat((results, net_out))
            # sum up batch loss
            loss = criterion(net_out, target)
            test_loss += loss
            # pred = net_out.data.max(1)[1]  # get the index of the max log-probability
            # correct += pred.eq(target.data).sum()
            
#         print('\nTest set: Average loss: {:.4f}'.format(
#          test_loss))

    print("test all dataset losss: ", test_loss.item())
    print("division by number of examples: ", len(test_loader.dataset))
    return results,  (test_loss.item() / len(test_loader.dataset)) * 100

In [10]:
def getFeatureSize(data, target):
    # Convert tensors to numpy
    data, target = data.cpu().detach().numpy(), target.cpu().detach().numpy()
    return data.shape[0], target.shape[0]

In [11]:
def denormalizeDataByMinMax(data, min_history, max_history):
    for i in range(data.shape[1]):
        min_val, max_val = min_history[i], max_history[i]
        data[:, i] = (data[:, i] * (max_val - min_val)) + min_val
    
    return data

In [12]:
def denormalizeDataByStdMean(data, std_history, mean_history):
    for i in range(data.shape[1]):
        std, mean = std_history[i], mean_history[i]
        data[:, i] = (data[:, i] * std) + mean
    
    return data

In [13]:
def generateClipAndFrameCols(results):
    clips = []
    frames = []
    loop_num = results.shape[0] // config.frame_num
    remaining_div = results.shape[0] % config.frame_num

    for i in range(loop_num):
        clips = np.concatenate((clips, np.full((1, config.frame_num), str(i))), axis=None)
        # clips = np.concatenate((clips, list(str(i) for i in range(config.frame_num))), axis=None)
        str_num_range = [*map(str, range(config.frame_num))]
        frames = np.concatenate((frames, str_num_range), axis=None)

    # Add the remaining rows, if the csv file not contains exactly rows num that divide by frame_num   
    if (remaining_div != 0):
        clips = np.concatenate((clips, np.full((1, remaining_div), str(loop_num))), axis=None)
        # clips = np.concatenate((clips, list(str(i) for i in range(remaining_div))), axis=None)
        str_num_range = [*map(str, range(remaining_div))]
        frames = np.concatenate((frames, str_num_range), axis=None)
    
    # Insert the clip & frame data to the results
    results = np.insert(results, 0, clips, axis=1)
    results = np.insert(results, 1, frames, axis=1)

    return results

In [14]:
def saveResultsToOutputFile(mode, filename, results):
    # Create the output csv columns
    blend_cols = []
    clip_and_frame_cols = ['clip', 'frame']
    quat_cols = ['Quaternion_x', 'Quaternion_y', 'Quaternion_z', 'Quaternion_w']
    for i in range(config.blend_range[0], config.blend_range[1]):
        blend_cols.append('Blendshape_{0}'.format(i))
    output_cols = clip_and_frame_cols + quat_cols + blend_cols
    
    # Convert the results to from tensor to numpy 
    results = results.cpu().detach().numpy()

    # Generate and insert clip & frame columns
    results = generateClipAndFrameCols(results)

    # Convert the results to data frame
    results = pd.DataFrame(results, columns=output_cols)

    # Convert columns data type from float to int
    results['clip'] = results['clip'].astype(int)
    results['frame'] = results['frame'].astype(int)
    
    # Assign values outside boundary to boundary values
    results.loc[:, quat_cols] = results.loc[:, quat_cols].clip(config.quat_domain[0], config.quat_domain[1])
    results.loc[:, blend_cols] = results.loc[:, blend_cols].clip(config.blend_domain[0], config.blend_domain[1])

    # Save the results to the output csv file
    file_path = f'{config.samples_path}\\{mode}\\{filename}'
    results.to_csv(file_path, index=False)
    print(f'[{datetime.now()}] Successfully saved the results to {file_path}')

In [15]:
if __name__ == "__main__":
    epochs=20
    batch_size=50
    log_interval=10
    learning_rate=1e-3
    
    device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    # Create the train data loader
    train_dataset, x_history, y_history = getDataset(Modes.TRAIN.value, device)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size,
        shuffle=True)
    # Create the test data loader
    test_dataset, _, _ = getDataset(Modes.TEST.value, device, x_history, y_history)
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size,
        shuffle=False)

    # Extract the input and output num of features
    in_num_of_features, out_num_of_features = getFeatureSize(*train_dataset[0])

    # Create the net
    net = Net(in_num_of_features, out_num_of_features)
    net.to(device)
    print(net)

    # Create an optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

    # Create a loss function
    criterion = nn.MSELoss(reduction='sum')
    # criterion = nn.L1Loss(reduction='mean')

    # Run the main training loop
    net = train(device, epochs, train_loader, net, optimizer, criterion, log_interval, test_loader)
                
    # Run a test loop
    results, test_loss = test(device, test_loader, net, criterion)
    
    # De-normalize the data to the original domains
    results = denormalizeDataByMinMax(results, *y_history)
    
    # Save the results to output file
    saveResultsToOutputFile(Modes.TEST.value, config.output_filename, results)

Sequential(
  (0): Linear(in_features=208, out_features=2048, bias=True)
  (1): PReLU(num_parameters=1)
  (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=2048, out_features=1024, bias=True)
  (4): PReLU(num_parameters=1)
  (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Dropout(p=0.3, inplace=False)
  (7): Linear(in_features=1024, out_features=512, bias=True)
  (8): PReLU(num_parameters=1)
  (9): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Dropout(p=0.3, inplace=False)
  (11): Linear(in_features=512, out_features=256, bias=True)
  (12): PReLU(num_parameters=1)
  (13): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): Dropout(p=0.3, inplace=False)
  (15): Linear(in_features=256, out_features=128, bias=True)
  (16): PReLU(num_parameters=1)
  (17): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=